**BASIC MNIST EXAMPLE WITH MLP**

Imports...

In [1]:
from __future__ import print_function

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import BatchNormalization as BN
from keras.layers import GaussianNoise as GN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout, Input, Reshape
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler as LRS
from keras.preprocessing.image import ImageDataGenerator


Init Plugin
Init Graph Optimizer
Init Kernel


Define batch size, number of epochs and number of classes

In [2]:
batch_size = 100
epochs = 75
num_classes=10

Load MNIST and transform



In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('training set', x_train.shape)
print('test set', x_test.shape)

x_train = x_train.reshape(60000, 28,28,1)
x_test = x_test.reshape(10000, 28,28,1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize [0..255]-->[0..1]
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

training set (60000, 28, 28)
test set (10000, 28, 28)


Define the NN topology, a sequential model with 2 hidden layers

In [4]:
model = Sequential()
model.add(Reshape(target_shape=(784,), input_shape=(28,28,1)))
model.add(GN(0.3))

model.add(Dense(1024))
model.add(BN())
model.add(GN(0.3))
model.add(Activation('relu'))

model.add(Dense(1024))
model.add(BN())
model.add(GN(0.3))
model.add(Activation('relu'))

model.add(Dense(1024))
model.add(BN())
model.add(GN(0.3))
model.add(Activation('relu'))

model.add(Dense(num_classes, activation='softmax'))

model.summary()


Metal device set to: Apple M1
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 784)               0         
_________________________________________________________________
module_wrapper (ModuleWrappe (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 1024)              803840    
_________________________________________________________________
batch_normalization (BatchNo (None, 1024)              4096      
_________________________________________________________________
module_wrapper_1 (ModuleWrap (None, 1024)              0         
_________________________________________________________________
activation (Activation)      (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)           

2021-12-20 17:12:30.860857: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-20 17:12:30.860939: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Define an optimizer 

In [5]:
sgd=SGD(learning_rate=0.01, decay=1e-6, momentum=0.9)

Compile the model, define loss and link the optimizer

In [6]:
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

Finally, train the model and evaluate over the test data

In [ ]:
datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=False
)

def scheduler(epoch):
    if epoch < 25:
        return .1
    elif epoch < 50:
        return 0.01
    else:
        return 0.001

history = model.fit(
    datagen.flow(x_train, y_train,batch_size=batch_size),             
    epochs=epochs,
    verbose=1,
    validation_data=(x_test, y_test),
    callbacks=[LRS(scheduler)]
)

# Evaluate over test
score = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])
print(f"Test error: {((1-score[1]) * 100):.3f}%")

2021-12-20 17:12:31.121677: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-20 17:12:31.121807: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-12-20 17:12:31.320205: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/75
598/600 [============================>.] - ETA: 0s - loss: 0.7280 - accuracy: 0.8023

2021-12-20 17:12:42.840072: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


600/600 [==============================] - 13s 21ms/step - loss: 0.7266 - accuracy: 0.8027 - val_loss: 0.1232 - val_accuracy: 0.9632
Epoch 2/75
600/600 [==============================] - 13s 21ms/step - loss: 0.3321 - accuracy: 0.8972 - val_loss: 0.0983 - val_accuracy: 0.9696
Epoch 3/75
600/600 [==============================] - 13s 21ms/step - loss: 0.2703 - accuracy: 0.9153 - val_loss: 0.0804 - val_accuracy: 0.9747
Epoch 4/75
600/600 [==============================] - 13s 21ms/step - loss: 0.2350 - accuracy: 0.9257 - val_loss: 0.0755 - val_accuracy: 0.9769
Epoch 5/75
600/600 [==============================] - 13s 21ms/step - loss: 0.2063 - accuracy: 0.9343 - val_loss: 0.0562 - val_accuracy: 0.9810
Epoch 6/75
600/600 [==============================] - 13s 21ms/step - loss: 0.1939 - accuracy: 0.9380 - val_loss: 0.0532 - val_accuracy: 0.9832
Epoch 7/75
600/600 [==============================] - 13s 21ms/step - loss: 0.1816 - accuracy: 0.9414 - val_loss: 0.0476 - val_accuracy: 0.9848
Epo

600/600 [==============================] - 13s 21ms/step - loss: 0.0719 - accuracy: 0.9765 - val_loss: 0.0230 - val_accuracy: 0.9919
Epoch 58/75
600/600 [==============================] - 13s 21ms/step - loss: 0.0762 - accuracy: 0.9757 - val_loss: 0.0227 - val_accuracy: 0.9920
Epoch 59/75
600/600 [==============================] - 13s 21ms/step - loss: 0.0731 - accuracy: 0.9758 - val_loss: 0.0232 - val_accuracy: 0.9918
Epoch 60/75
600/600 [==============================] - 13s 21ms/step - loss: 0.0737 - accuracy: 0.9763 - val_loss: 0.0230 - val_accuracy: 0.9919
Epoch 61/75
600/600 [==============================] - 13s 21ms/step - loss: 0.0743 - accuracy: 0.9755 - val_loss: 0.0227 - val_accuracy: 0.9922
Epoch 62/75
600/600 [==============================] - 13s 21ms/step - loss: 0.0740 - accuracy: 0.9766 - val_loss: 0.0229 - val_accuracy: 0.9920
Epoch 63/75
600/600 [==============================] - 13s 21ms/step - loss: 0.0733 - accuracy: 0.9767 - val_loss: 0.0225 - val_accuracy: 0.99

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()